# 🚀 机器学习量化策略入门实践

这个笔记本将带你用 **最简化的代码** 理解整个流程。

**目标**: 在50行代码内完成一个完整的机器学习策略

---

## 学习流程

1. 加载数据 → 2. 定义标签 → 3. 创建特征 → 4. 训练模型 → 5. 评估结果


## Step 1: 加载数据

从数据库读取一个期货品种的历史数据


In [ ]:
import pandas as pd
import numpy as np
import sqlite3
from pathlib import Path
import matplotlib.pyplot as plt

# 设置中文显示
plt.rcParams['font.sans-serif'] = ['Arial Unicode MS', 'SimHei']
plt.rcParams['axes.unicode_minus'] = False

# 连接数据库
db_path = Path('../..') / 'database' / 'futures' / 'futures.db'
conn = sqlite3.connect(db_path)

# 读取螺纹钢数据
df = pd.read_sql_query("SELECT * FROM hist_rb", conn)
conn.close()

# 整理数据
df = df.rename(columns={
    'trade_date': 'date', 'open_price': 'open', 'high_price': 'high',
    'low_price': 'low', 'close_price': 'close'
})
df['date'] = pd.to_datetime(df['date'])
df = df.sort_values('date').reset_index(drop=True)

print(f"数据形状: {df.shape}")
print(f"日期范围: {df['date'].min().date()} ~ {df['date'].max().date()}")
df.tail()


## Step 2: 定义标签 (最关键!)

**问题**: 什么是"好的交易机会"？

**我们的定义**: 如果今天买入，未来5天内最高价涨幅 ≥ 3%，就是好机会 (label=1)


In [ ]:
# 定义参数
FUTURE_DAYS = 5    # 看未来5天
THRESHOLD = 0.03   # 3%的涨幅阈值

# 计算未来5天的最高价
df['future_high'] = df['high'].shift(-1).rolling(FUTURE_DAYS).max().shift(-FUTURE_DAYS + 1)

# 最大上涨幅度
df['max_up_return'] = (df['future_high'] - df['close']) / df['close']

# 生成标签
df['label'] = (df['max_up_return'] >= THRESHOLD).astype(int)

# 统计
print("标签分布:")
print(df['label'].value_counts())
print(f"\n正样本(好机会)占比: {df['label'].mean()*100:.2f}%")


## Step 3: 创建特征

用技术指标来描述当前市场状态


In [ ]:
# 创建5个核心特征
close = df['close']
high = df['high']
low = df['low']
volume = df['volume']

# 1. 动量: 过去N天收益率
df['feat_ret_5'] = close.pct_change(5)
df['feat_ret_20'] = close.pct_change(20)

# 2. 均线偏离: 价格相对均线的位置
ma_20 = close.rolling(20).mean()
df['feat_ma20_dev'] = (close - ma_20) / ma_20

# 3. 波动率: 价格波动程度
df['feat_volatility'] = close.pct_change().rolling(20).std()

# 4. 价格位置: 在近期高低点区间的位置 (0=最低, 1=最高)
high_20 = high.rolling(20).max()
low_20 = low.rolling(20).min()
df['feat_price_pos'] = (close - low_20) / (high_20 - low_20 + 1e-6)

# 5. 成交量比: 相对平均成交量
vol_ma = volume.rolling(20).mean()
df['feat_vol_ratio'] = volume / (vol_ma + 1e-6)

# 查看特征
feature_cols = [col for col in df.columns if col.startswith('feat_')]
print(f"创建了 {len(feature_cols)} 个特征: {feature_cols}")


## Step 4: 划分数据 & 训练模型

按时间划分: 2022年前训练，2023年验证，2024年测试


In [ ]:
# 移除缺失值
df_clean = df.dropna(subset=feature_cols + ['label']).copy()

# 按时间划分
df_train = df_clean[df_clean['date'] <= '2022-12-31']
df_valid = df_clean[(df_clean['date'] > '2022-12-31') & (df_clean['date'] <= '2023-12-31')]
df_test = df_clean[df_clean['date'] > '2023-12-31']

print(f"训练集: {len(df_train)} 样本")
print(f"验证集: {len(df_valid)} 样本")
print(f"测试集: {len(df_test)} 样本")


In [ ]:
# 训练模型
try:
    import lightgbm as lgb
    model = lgb.LGBMClassifier(n_estimators=100, max_depth=5, verbosity=-1)
except ImportError:
    from sklearn.ensemble import GradientBoostingClassifier
    model = GradientBoostingClassifier(n_estimators=100, max_depth=5)

# 准备数据
X_train, y_train = df_train[feature_cols].values, df_train['label'].values
X_valid, y_valid = df_valid[feature_cols].values, df_valid['label'].values
X_test, y_test = df_test[feature_cols].values, df_test['label'].values

# 训练
model.fit(X_train, y_train)
print("✅ 模型训练完成!")


## Step 5: 评估模型

用几个关键指标来判断模型好不好


In [ ]:
from sklearn.metrics import roc_auc_score, precision_score, recall_score

# 预测概率
y_pred_proba = model.predict_proba(X_test)[:, 1]

# 用0.5作为阈值分类
y_pred = (y_pred_proba >= 0.5).astype(int)

# 计算指标
auc = roc_auc_score(y_test, y_pred_proba)
precision = precision_score(y_test, y_pred, zero_division=0)
recall = recall_score(y_test, y_pred, zero_division=0)

print("=" * 50)
print("📊 模型评估结果 (测试集)")
print("=" * 50)
print(f"""
AUC: {auc:.4f}
  → 解读: 0.5=随机, 0.6=一般, 0.7=不错, 0.8=很好

精确率 (Precision): {precision*100:.2f}%
  → 模型说"买"的里面，有多少真的涨了?

召回率 (Recall): {recall*100:.2f}%
  → 所有涨的机会中，模型抓住了多少?
""")


## Step 6: 可视化理解

看看模型预测的概率分布


In [ ]:
fig, axes = plt.subplots(1, 2, figsize=(12, 4))

# 左图: 概率分布
ax1 = axes[0]
ax1.hist(y_pred_proba[y_test==0], bins=30, alpha=0.5, label='实际=0 (普通)', density=True)
ax1.hist(y_pred_proba[y_test==1], bins=30, alpha=0.5, label='实际=1 (好机会)', density=True)
ax1.set_xlabel('预测概率')
ax1.set_ylabel('密度')
ax1.set_title('预测概率分布')
ax1.legend()
ax1.axvline(x=0.5, color='red', linestyle='--', alpha=0.5, label='阈值=0.5')

# 右图: 特征重要性
ax2 = axes[1]
if hasattr(model, 'feature_importances_'):
    importance = model.feature_importances_
    sorted_idx = np.argsort(importance)
    ax2.barh(range(len(sorted_idx)), importance[sorted_idx])
    ax2.set_yticks(range(len(sorted_idx)))
    ax2.set_yticklabels([feature_cols[i] for i in sorted_idx])
    ax2.set_xlabel('重要性')
    ax2.set_title('特征重要性')

plt.tight_layout()
plt.show()

print("💡 特征重要性告诉你：模型认为哪些指标对预测最有用")


---
## 🎯 练习: 动手试试

### 练习1: 修改阈值，观察正样本比例变化


In [ ]:
# 尝试不同的阈值
print("不同阈值下的正样本比例:")
print("-" * 40)
for thresh in [0.01, 0.02, 0.03, 0.04, 0.05, 0.08, 0.10]:
    label = (df_clean['max_up_return'] >= thresh)
    ratio = label.mean()
    print(f"阈值 {thresh*100:4.0f}%: 正样本比例 = {ratio*100:5.2f}%")

print("\n💡 阈值越高 → 正样本越少 → 标准越严格")


### 练习2: 添加一个新特征 - RSI


In [ ]:
# RSI (相对强弱指标) 的计算方法
# RSI = 100 - 100 / (1 + RS)
# RS = 平均上涨 / 平均下跌

delta = df_clean['close'].diff()
gain = delta.where(delta > 0, 0)
loss = (-delta).where(delta < 0, 0)

avg_gain = gain.rolling(14).mean()
avg_loss = loss.rolling(14).mean()

rs = avg_gain / (avg_loss + 1e-6)
df_clean['feat_rsi'] = 100 - (100 / (1 + rs))

print("RSI 统计:")
print(df_clean['feat_rsi'].describe())
print("\n💡 RSI < 30 通常被认为是超卖（可能反弹）")
print("💡 RSI > 70 通常被认为是超买（可能回调）")


---
## 🎓 总结

恭喜你完成了入门实践！现在你应该理解了：

| 步骤 | 关键概念 | 对应完整代码 |
|------|---------|-------------|
| 1. 数据 | K线数据、OHLCV | `load_history_from_db()` |
| 2. 标签 | 未来最大涨幅定义好机会 | `assign_labels_v2()` |
| 3. 特征 | 技术指标描述市场状态 | `make_features_v2()` |
| 4. 模型 | LightGBM 学习特征-标签关系 | `train_model()` |
| 5. 评估 | AUC、精确率、召回率 | `analyze_backtest_results()` |

### 下一步学习

1. 阅读完整的 `学习指南_机器学习量化策略.md`
2. 运行完整的 `futures_trend_ml.py` 并理解输出
3. 尝试修改参数观察结果变化

有问题随时问！🚀
